# İÇİNDEKİLER
>## 1. Veri Madenciliği Uygulama Yerinin Bulunması ve Veri Setinin Temini
>## 2. Modelleme
>>### 2.1. Araştırma Hedefini Belirlenmesi (Problem / Araştırma Sorusu)
>>### 2.2. Analizde Kullanılacak Verisetinin İncelenmesi Ve Veri Önişleme
>## 3. Test Ve Eğitim Veri Setlerinin Hazırlanması
>## 4. Sınıflandırılma Algoritmalarının Uygulanması ve Performans Analizleri
>>### 4.1. Logistic Regression
>>### 4.2. Naive Bayes
>>>#### 4.2.1. GaussianNB
>>>#### 4.2.2. MultinomialNB 
>>>#### 4.2.3. BernoulliNB
>## 5.Sınıflandırma Algoritmalarının Performanslarının Kıyaslanması
>## 6.Sonuç Raporu

# 1. Veri Madenciliği Uygulama Yerinin Bulunması ve Veri Setinin Temini
Bu projede Veri Madenciliği tahminleyici yöntemlerden sınıflandırma üzerine bir çalışma yapılacaktır. Uygulamamız bir telekomünikasyon firmasının üzerinde olacak ve analiz için gerekli veri setinin temini büyük çaplı verilerin firmalar tarafından yarışma usulünce kullanıcılara açıldığı kaggle.com adlı siteden olmuştur.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df=pd.read_csv("../input/telco-customer-churn/WA_Fn-UseC_-Telco-Customer-Churn.csv",sep=',',decimal='.')

In [ ]:
df.rename(columns={'customerID':'Müşteri ID','gender':'Cinsiyet','SeniorCitizen':'Yaşlı','Partner':'Evli',
                   'Dependents':'Ekonomik Bağımlı','tenure':'Abonelik Süresi','PhoneService':'Telefon Hizmeti',
                   'MultipleLines':'Birden Fazla Hat','InternetService':'İnternet Servisi',
                   'OnlineSecurity':'Çevrimiçi Güvenlik','OnlineBackup':'Çevrimiçi Yedekleme',
                   'DeviceProtection':'Cihaz Koruma','TechSupport':'Teknik Destek','StreamingTV':'Televizyon',
                   'StreamingMovies':'Film','Contract':'Sözleşme Süresi','PaperlessBilling':'Çevrimiçi Fatura',
                   'PaymentMethod':'Ödeme Yöntemi','MonthlyCharges':'Aylık Ödeme','TotalCharges':'Toplam Ödeme',
                   'Churn':'Müşteri Kaybı'},inplace=True)

# 2. Modelleme


## 2.1. Araştırma Hedefini Belirlenmesi (Problem / Araştırma Sorusu)
Yeni müşteri elde etme maliyeti mevcut müşteriyi elde tutma maliyetinden daha yüksektir hal böyleyken şirketler önce mevcut müşterilerini korumalıdır. Bunun için de kaybetme tehlikesi oluşan müşteriler için önlemler almak gereklidir. Burada devreye insan davranış faktörü girdiğinden nitelikte yeterli müşteri verisine sahip iseniz sınıflandırma analiziyle bu potansiyele sahip müşterileri şirket kayıtlarından önceden tespit edebilir buna göre önlemler alabilirsiniz. 
Bu bağlamda gerçekleştireceğimiz projemizde,<br>
Araştırma sorumuz,<br>
•	 Müşteri kaybı, oluşmadan tespit edilebilir mi?<br>
İle bu araştırmamızda hedeflediğimiz,<br>
•	Sadık müşteri kitlesinin oluşturulması.<br>
Şeklinde olmuştur.<br>
Analiz genel olarak şu şekilde işleyecek,<br>
Telcom şirketinin elinde yeterli sayıda veri bulunmaktadır, müşteriler belirli kriterlere göre veri tabanına kayıtlıdır buradan çekilen veri setiyle müşteri kaybı analizi yapılacaktır.<br>
Müşteri Kaybı analizi bu tip şirketler için çok önemlidir müşterileri başka firmalara kaybetmemelidirler. Bu bağlamda yapılacak analiz genel olarak şu şekilde işleyecek ay sonu müşterilere ait nitelikler ve kayıp oluşup oluşmadığı durumu bilgisi üzerinden eğitim verisi oluşturulup test edilecek ve gelecek ay müşterilerin nitelik durumlarına göre kayıp olma durumları hemen aybaşından bu eğitilen modelle tespit edilecektir. Tespit edilen potansiyel kayıp müşterilerin elden kaçırılmaması için şirket politikaları geliştirilecektir. Böylece müşteri kaybı azaltılacak bunun yanında da rakip firmanın müşteri kazanması önlenmiş olacaktır.

## 2.2. Analizde Kullanılacak Verisetinin İncelenmesi Ve Veri Önişleme
Anazlide kullanılmak üzere 21 öznitelikten oluşan bir müşteri veri seti temin edilmiştir, bu özniteliklerden hedef değişken “Müşteri Kaybı(Churn)” şeklinde tanımlanmıştır.

Veri setimiz 7043 satır, 21 sütundan oluşmaktadır.

In [ ]:
df.head(8000)

CostumerID(Müşteri ID) kısmı gereksiz olduğu için veri setimizden çıkaralım.

In [ ]:
df.drop('Müşteri ID', axis=1, inplace=True)

Grafik okurken kafa karıştırmaması için hedef sütunumuzdaki verileri türkçeleştirelim.

In [ ]:
df["Müşteri Kaybı"]= df["Müşteri Kaybı"].replace("No","Müşteri Kaybı Oluşmadı") 
df["Müşteri Kaybı"]= df["Müşteri Kaybı"].replace("Yes","Müşteri Kaybı Oluştu")

Yaşlı sütununun verilerini kategorik hale getirelim.

In [ ]:
df["Yaşlı"]= df["Yaşlı"].replace(0, "No") 
df["Yaşlı"]= df["Yaşlı"].replace(1, "Yes") 

In [ ]:
df.info()

Görüldüğü gibi "Toplam Ödeme" kısmı float olması gerekirken object olarak belirlenmiş.Onu düzeltelim.

In [ ]:
df['Toplam Ödeme'] = pd.to_numeric(df['Toplam Ödeme'], errors='coerce')
df['Toplam Ödeme'] = df['Toplam Ödeme'].fillna(value=0)

Yaşlı sütununda yaptığımız değişiklikten ötürü sütunun data tipi de değişmeli.

In [ ]:
df['Yaşlı'] = df['Yaşlı'].astype('object')

Görüldüğü üzere integer ve float tipi verileri içeren sütunlarda istatistiksel veriler getirildi.
Count değerleri eşit olduğuna göre aykırı veri bulunmamaktadır.

In [ ]:
df.describe()

Kayıp müşterilerin kutu grafiği oluşturularak kayıp olan ve olmayan müşteri sayıları görselleştirildi.

In [ ]:
sns.countplot(x = "Müşteri Kaybı", data = df)
df.loc[:, 'Müşteri Kaybı'].value_counts()

Kayıp veri bulunmamaktadır.

In [ ]:
df.isnull().sum()

Veriler kategorik ve sayısal olarak sınıflandırılmıştır.
Hedef değişkenimiz olan Kayıp, kategorik alanlara dahil edilmeyecektir.

In [ ]:
Kategorik = df.select_dtypes(include='object').drop('Müşteri Kaybı', axis=1).columns.tolist()
Sayısal = df.select_dtypes(exclude='object').columns.tolist()

Sınıflandırılan kategorik değişkenlere göz gezdirildi.

In [ ]:
for c in Kategorik:
    print('Column {} unique values: {}'.format(c, len(df[c].unique())))

Hedef değişkenin diğer değişkenlerle olan ilişkisi hakkında ön fikir sahibi olmak için kategorik ve sayısal verileri<br> grafikleştirelim.

Kategorik değişkenler kutu grafiği, sayısal değişkenler çizgi grafiği olarak görselleştirildi.

In [ ]:
plt.figure(figsize=(18,18))
for i,c in enumerate(Kategorik):
    plt.subplot(5,4,i+1)
    sns.countplot(df[c], hue=df['Müşteri Kaybı'])
    plt.title(c)
    plt.xlabel('')

In [ ]:
plt.figure(figsize=(20,5))
for i,c in enumerate(['Abonelik Süresi', 'Aylık Ödeme', 'Toplam Ödeme']):
    plt.subplot(1,3,i+1)
    sns.distplot(df[df['Müşteri Kaybı'] == 'Müşteri Kaybı Oluşmadı'][c], kde=True, color='blue', hist=False, kde_kws=dict(linewidth=2), label='Müşteri Kaybı Oluşmadı')
    sns.distplot(df[df['Müşteri Kaybı'] == 'Müşteri Kaybı Oluştu'][c], kde=True, color='Orange', hist=False, kde_kws=dict(linewidth=2), label='Müşteri Kaybı Oluştu')
    plt.title(c)

Şuana kadar işlediğimiz verileri kontrol edelim.

In [ ]:
df.head(8000)

In [ ]:
df.info()

Eksik verimiz olup olmadığına bakmıştık şimdi aykırı verileri gözlemleyelim.

In [ ]:
import seaborn as sns
sns.boxplot(x=df['Abonelik Süresi'],y=df['Müşteri Kaybı'])

In [ ]:
import seaborn as sns
sns.boxplot(x=df['Aylık Ödeme'],y=df['Müşteri Kaybı'])

In [ ]:
import seaborn as sns
sns.boxplot(x=df['Toplam Ödeme'],y=df['Müşteri Kaybı'])

Toplam ödemenin müşteri kaybı oluşan kısmındayoğun şekilde aykırı veriler görüyoruz. Aslında bu durum bile bize müşteri kayıplarının neden oluştuğunu gösteriyor fakat yinede daha detaylı analiz yapalım.Bunun haricinde abonelik süresinde de bir miktar aykırı veri görünüyor. Şimdi aykırı verilerimi temizleyelim.Öncelikle veriler labelencoder ile tamamen sayısal hale getirilecek.

In [ ]:
from sklearn.preprocessing import LabelEncoder
encoded = df.apply(lambda x: LabelEncoder().fit_transform(x) if x.dtype == 'object' else x)
encoded.head(8000)

In [ ]:
Müşteri_Kaybı_Yaşandı=encoded.loc[encoded['Müşteri Kaybı'].abs()>0]
Müşteri_Kaybı_Yaşandı

In [ ]:
Q1 = Müşteri_Kaybı_Yaşandı['Toplam Ödeme'].quantile(0.25)
Q3 = Müşteri_Kaybı_Yaşandı['Toplam Ödeme'].quantile(0.75)
IQR = Q3 - Q1
IQR

In [ ]:
Q=Q3+(1.5*IQR)
Q

In [ ]:
encoded_out = encoded[~((encoded['Toplam Ödeme'] < (Q3 + 1.5 * IQR)))&(encoded['Müşteri Kaybı']>0)]
encoded_out.head(8000)

Aykırı veriler çağırıldı.Grafikte gözlemlediğimiz yoğunluktaki aykırı veri sayısı 109'muş.

In [ ]:
encoded.drop(encoded[~((encoded['Toplam Ödeme'] < (Q3 + 1.5 * IQR)))&(encoded['Müşteri Kaybı']>0)].index, inplace=True)
encoded.head(8000)

Toplam ödeme kısmındaki aykırı veriler silindi.Sıra Abonelik Süresinde.

In [ ]:
Q1_A = Müşteri_Kaybı_Yaşandı['Abonelik Süresi'].quantile(0.25)
Q3_A = Müşteri_Kaybı_Yaşandı['Abonelik Süresi'].quantile(0.75)
IQR_A = Q3_A - Q1_A
IQR_A

In [ ]:
Q_A=Q3_A+(1.5*IQR_A)
Q_A

In [ ]:
encoded_A_out = encoded[~((encoded['Abonelik Süresi'] < (Q3_A + 1.5 * IQR_A)))&(encoded['Müşteri Kaybı']>0)]
encoded_A_out.head(8000)

Abonelik süresindeki aykırı veriler getirildi.

In [ ]:
encoded.drop(encoded[~((encoded['Abonelik Süresi'] < (Q3_A + 1.5 * IQR_A)))&(encoded['Müşteri Kaybı']>0)].index, inplace=True)
encoded.head(8000)

Aykırı verilerinde tamamen temizlenmesi ile veri ön işleme tamamlandı. Fakat uygulayacağımız testlere göre farklı işlemelerde yapacağız.

# 3. Test ve Eğitim Verilerinin Hazırlanması

Bu aşamada hedef değişkeni tanımlanan ve algoritma öncesi son halini alan verinin test ve eğitim olarak ikiye ayrılma işlemidir. Bu işlem için uygun olan ayırma eşiği 85-15 olarak uygun görülmüştür.

Genel bir gösterim olması amacıyla aşağıda bir test ve eğitim verilerinin ayrılması işlemlerini göstereceğim.Test algoritmalarını uygulamadan önce teste göre işlemler yaptıktan sonra bu adımlar uygulanacak.

In [ ]:
x = df.drop('Müşteri Kaybı', axis = 1)              
y = df['Müşteri Kaybı'] 

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.85, random_state = 400)

In [ ]:
x_test.head(8000)

In [ ]:
x_train.head(8000)

In [ ]:
y_test.head(8000)

In [ ]:
y_train.head(8000)

# 4. Sınıflandırılma Algoritmalarının Uygulanması ve Performans Analizleri
Müşteri kayıplarının önceden tespitini sınıflandırma algoritmalarıyla yapacağız.
Uygulamamızda dört sınıflandırma algoritması veriye uygulanacak ve performans çıktıları analiz edilecektir.Bu performans çıktıları ise 5.bölümde kıyaslanarak, kullanılacak olan sınıflandıma algoritması belirlenecektir.<br>
Bu dört sınıflandırma algoritması;<br>
Lojistik Regresyon<br>
Naive Bayes<br>
Karar Ağacı<br>
K-NN(En Yakın Komşu)<br>

## 4.1. Logistic Regression
Lojistik regresyon, çıktı değişkeninin kesikli değerler aldığı durumlarda geçerlidir.<br>
Verilerimizin içinde ikiden çok seçeneği olan sütunlar olduğu için çoklu lojistik regresyon kullanacağız.

Lojistik regresyona başlamadan önce küçük bir korelasyon analizi yaparak işimize yaramayan sütunları atalım.

In [ ]:
x=encoded['Cinsiyet']
y=encoded['Müşteri Kaybı']
print('Cinsiyet:', x.corr(y)*100)
x=encoded['Yaşlı']
y=encoded['Müşteri Kaybı']
print('Yaşlı:', x.corr(y)*100)
x=encoded['Evli']
y=encoded['Müşteri Kaybı']
print('Evli:', x.corr(y)*100)
x=encoded['Ekonomik Bağımlı']
y=encoded['Müşteri Kaybı']
print('Ekonomik Bağımlı:', x.corr(y)*100)
x=encoded['Abonelik Süresi']
y=encoded['Müşteri Kaybı']
print('Abonelik Süresi:', x.corr(y)*100)
x=encoded['Telefon Hizmeti']
y=encoded['Müşteri Kaybı']
print('Telefon Hizmeti:', x.corr(y)*100)
x=encoded['Birden Fazla Hat']
y=encoded['Müşteri Kaybı']
print('Birden Fazla Hat:', x.corr(y)*100)
x=encoded['İnternet Servisi']
y=encoded['Müşteri Kaybı']
print('İnternet Servisi:', x.corr(y)*100)
x=encoded['Çevrimiçi Güvenlik']
y=encoded['Müşteri Kaybı']
print('Çevrimiçi Güvenlik:', x.corr(y)*100)
x=encoded['Çevrimiçi Yedekleme']
y=encoded['Müşteri Kaybı']
print('Çevrimiçi Yedekleme:', x.corr(y)*100)
x=encoded['Cihaz Koruma']
y=encoded['Müşteri Kaybı']
print('Cihaz Koruma:', x.corr(y)*100)
x=encoded['Teknik Destek']
y=encoded['Müşteri Kaybı']
print('Teknik Destek:', x.corr(y)*100)
x=encoded['Televizyon']
y=encoded['Müşteri Kaybı']
print('Televizyon:', x.corr(y)*100)
x=encoded['Film']
y=encoded['Müşteri Kaybı']
print('Film:', x.corr(y)*100)
x=encoded['Sözleşme Süresi']
y=encoded['Müşteri Kaybı']
print('Sözleşme Süresi:', x.corr(y)*100)
x=encoded['Çevrimiçi Fatura']
y=encoded['Müşteri Kaybı']
print('Çevrimiçi Fatura:', x.corr(y)*100)
x=encoded['Ödeme Yöntemi']
y=encoded['Müşteri Kaybı']
print('Ödeme Yöntemi:', x.corr(y)*100)
x=encoded['Aylık Ödeme']
y=encoded['Müşteri Kaybı']
print('Aylık Ödeme:', x.corr(y)*100)
x=encoded['Toplam Ödeme']
y=encoded['Müşteri Kaybı']
print('Toplam Ödeme:', x.corr(y)*100)

In [ ]:
encoded.drop('Cinsiyet', axis=1, inplace=True)
encoded.drop('Telefon Hizmeti', axis=1, inplace=True)
encoded.drop('Birden Fazla Hat', axis=1, inplace=True)
encoded.drop('İnternet Servisi', axis=1, inplace=True)
encoded.drop('Televizyon', axis=1, inplace=True)
encoded.drop('Film', axis=1, inplace=True)

Korelasyon sonucu 10 dan düşük olan sütunları attık.Korelasyon sonucunda da sözleşme süresinin ve abonelik süresinin müşteri kaybına etkisini görüyoruz.

In [ ]:
x=encoded.drop('Müşteri Kaybı',axis=1)
y=encoded['Müşteri Kaybı']

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.85, random_state=43)

In [ ]:
from sklearn.preprocessing import MinMaxScaler,StandardScaler

In [ ]:
sc=StandardScaler()

In [ ]:
x_train=sc.fit_transform(x_train)
x_test=sc.fit_transform(x_test)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder

multinomial lojistik resresyon, solver olarak sadece newton-cg ve lbfgs yi desteklemektedir.Bu durumda penalty=l2 mecburi seçenektir.

In [ ]:
Logistic_Regression = LogisticRegression(C=0.5,tol=0.1,multi_class='multinomial',solver='newton-cg',penalty='l2',max_iter=100)
Logistic_Regression.fit(x_train, y_train)

In [ ]:
y_pred=Logistic_Regression.predict(x_test)

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import accuracy_score

In [ ]:
classification_report(y_true=y_test, y_pred=y_pred)

In [ ]:
accuracy_score(y_test, y_pred)*100

In [ ]:
confusion_matrix(y_test, y_pred)

## 4.2. Naive Bayes
Algoritmanın çalışma şekli bir eleman için her durumun olasılığını hesaplar ve olasılık değeri en yüksek olana göre sınıflandırır.<br>
Naive Bayes, 3 gruba ayrılır;<br>
GaussianNB<br>
MultinomialNB<br>
BernoulliNB<br>

### 4.2.1. GaussianNB

In [ ]:
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
NBG = GaussianNB()
NBG.fit(x_train, y_train)
y_tahmin = NBG.predict(x_test)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.85, random_state=42)
print("x_train: ", x_train.shape)
print("x_test: ", x_test.shape)
print("y_train: ", y_train.shape)
print("y_test: ", y_test.shape)
print("Navy Bayes Gaussian Doğruluk Skoru :",accuracy_score(y_test, y_tahmin))
print("Karışıklık Matrisi :",confusion_matrix(y_test, y_tahmin))
print("Sınıflandırma Raporu :",classification_report(y_test, y_tahmin))

### 4.2.2. MultinomialNB 


In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
NBM = MultinomialNB()
NBM.fit(x_train, y_train)
y_tahmin = NBM.predict(x_test)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.85, random_state=42)
print("x_train: ", x_train.shape)
print("x_test: ", x_test.shape)
print("y_train: ", y_train.shape)
print("y_test: ", y_test.shape)
print("Navy Bayes Multinomial Doğruluk Skoru :",accuracy_score(y_test, y_tahmin))
print("Karışıklık Matrisi :",confusion_matrix(y_test, y_tahmin))
print("Sınıflandırma Raporu :",classification_report(y_test, y_tahmin))

### 4.2.3. BernoulliNB

In [ ]:
from sklearn.naive_bayes import BernoulliNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
NBB = BernoulliNB()
NBB.fit(x_train, y_train)
y_tahmin = NBB.predict(x_test)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.85, random_state=42)
print("x_train: ", x_train.shape)
print("x_test: ", x_test.shape)
print("y_train: ", y_train.shape)
print("y_test: ", y_test.shape)
print("Navy Bayes Bernoulli Doğruluk Skoru :",accuracy_score(y_test, y_tahmin))
print("Karışıklık Matrisi :",confusion_matrix(y_test, y_tahmin))
print("Sınıflandırma Raporu :",classification_report(y_test, y_tahmin))

## 4.2. Karar Ağacı
Sınıflama, özellik ve hedefe göre karar düğümleri ve yaprak düğümlerinden oluşan ağaç yapısı formunda model oluşturan bir sınıflandırma yöntemidir

In [ ]:
from sklearn.preprocessing import KBinsDiscretizer  
est = KBinsDiscretizer(n_bins=4, encode='ordinal', strategy='quantile')
encoded['Abonelik Süresi'] = est.fit_transform(encoded['Abonelik Süresi'].values.reshape(-1,1))

In [ ]:
from sklearn.preprocessing import KBinsDiscretizer  
est = KBinsDiscretizer(n_bins=4, encode='ordinal', strategy='quantile')
encoded['Aylık Ödeme'] = est.fit_transform(encoded['Aylık Ödeme'].values.reshape(-1,1))

In [ ]:
from sklearn.preprocessing import KBinsDiscretizer  
est = KBinsDiscretizer(n_bins=4, encode='ordinal', strategy='quantile')
encoded['Toplam Ödeme'] = est.fit_transform(encoded['Toplam Ödeme'].values.reshape(-1,1))

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score 
from sklearn.metrics import confusion_matrix as KarışıklıkMatrisi

x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.85, random_state=42)
Karar_Ağacı = DecisionTreeClassifier(max_depth = 4, random_state=42)
Karar_Ağacı.fit(x_train, y_train)

In [ ]:
predictions = Karar_Ağacı.predict(x_test)
score = round(accuracy_score(y_test, predictions), 2)
KarışıklıkMatrisi = KarışıklıkMatrisi(y_test, predictions)
sns.heatmap(KarışıklıkMatrisi, annot=True, fmt=".0f")
plt.xlabel('Tahmin Değerler')
plt.ylabel('Gerçek Veriler')
plt.title('Doğruluk Skoru: {0}'.format(score), size = 15)
plt.show()

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, predictions, target_names=['Kayıp Olmayanlar', 'Kayıp Olanlar']))

In [ ]:
from sklearn import tree
clf = tree.DecisionTreeClassifier(max_depth = 4, random_state=42,min_weight_fraction_leaf=0.0)
clf = clf.fit(x, y)

In [ ]:
tree.plot_tree(clf,fontsize=10) 

In [ ]:
plt.figure(figsize=(16, 9))

from sklearn import ensemble

Karar_Ağacı2 = DecisionTreeClassifier(max_depth = 4, random_state=42)
Karar_Ağacı2.fit(x_train, y_train)
ranking = Karar_Ağacı2.feature_importances_
features = np.argsort(ranking)[::-1][:10]
columns = x.columns

plt.title("Modelin Özniteliklerinin Önem sıralamasının Analiz Edilmesi", y = 1.03, size = 18)
plt.bar(range(len(features)), ranking[features], color="lime", align="center")
plt.xticks(range(len(features)), columns[features], rotation=80)
plt.show()

## 4.3. KNN(En Yakın Komşu)
Parametre olarak sonradan belirlenen bir gözlem değeri olan k değeri uzaklıklarının ve en küçük uzaklığa sahip k sayıda gözlemin seçilmesi esasına dayanmaktadır. 

K değerini en iyi şekilde atamak için kullanılan yöntem.

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
error = []

# Calculating error for K values between 1 and 40
for i in range(1, 40):
    knn = KNeighborsClassifier(n_neighbors=i)
    knn.fit(x_train, y_train)
    pred_i = knn.predict(x_test)
    error.append(np.mean(pred_i != y_test))

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(range(1, 40), error, color='red', linestyle='dashed', marker='o',markerfacecolor='blue', markersize=10)
plt.title('Hata Oranı K İçin')
plt.xlabel('K Değeri')
plt.ylabel('Ortalama Hata')

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
KNN = KNeighborsClassifier(n_neighbors=1)
KNN.fit(x_train, y_train)

In [ ]:
y_pred = KNN.predict(x_test)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

Anlamlılık düzeyi 0,73 çıktı.

# 5.Sınıflandırma Algoritmalarının Performanslarının Kıyaslanması
Sınıflandırma Modelinin Performansı Karışıklık Matrisi (Confusion Matrix) üzerinden detaylı olarak ölçülebilir.<br>
Ancak biz sadece auc-roc grafiği kullanacağız.


In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from matplotlib import pyplot
from sklearn.datasets import make_classification
x, y = make_classification(n_samples=1000, n_classes=2, random_state=1)
trainx, testx, trainy, testy = train_test_split(x, y, test_size=0.85, random_state=42)
KNN_tahmin = [0 for _ in range(len(testy))]
NBG_tahmin = [0 for _ in range(len(testy))]
Karar_Ağacı_tahmin = [0 for _ in range(len(testy))]
Logistic_Regression_tahmin = [0 for _ in range(len(testy))]

model = NBG
model.fit(trainx, trainy)
model2 = KNN
model2.fit(trainx, trainy)
model3=Karar_Ağacı
model3.fit(trainx, trainy)
model4=Logistic_Regression
model4.fit(trainx, trainy)

NBG_tahmin = model.predict_proba(testx)
KNN_tahmin = model2.predict_proba(testx)
Karar_Ağacı_tahmin = model3.predict_proba(testx)
Logistic_Regression_tahmin= model4.predict_proba(testx)

NBG_tahmin = NBG_tahmin[:, 1]
KNN_tahmin = KNN_tahmin[:, 1]
Karar_Ağacı_tahmin = Karar_Ağacı_tahmin[:, 1]
Logistic_Regression_tahmin = Logistic_Regression_tahmin[:, 1]

KNN_hassasiyet = roc_auc_score(testy, KNN_tahmin)
NBG_hassasiyet = roc_auc_score(testy, NBG_tahmin)
Karar_Ağacı_hassasiyet = roc_auc_score(testy, Karar_Ağacı_tahmin)
Logistic_Regression_hassasiyet = roc_auc_score(testy, Logistic_Regression_tahmin)

print('KNN: ROC AUC=%.3f' % (KNN_hassasiyet))
print('Navy Bayes Gaussian: ROC AUC=%.3f' % (NBG_hassasiyet))
print('Karar Ağacı: ROC AUC=%.3f' % (Karar_Ağacı_hassasiyet))
print('Logistic Regresyon: ROC AUC=%.3f' % (Logistic_Regression_hassasiyet))

KNN_fpr, KNN_tpr, _ = roc_curve(testy, KNN_tahmin)
NBG_fpr, NBG_tpr, _ = roc_curve(testy, NBG_tahmin)
Karar_Ağacı_fpr, Karar_Ağacı_tpr, _ = roc_curve(testy, Karar_Ağacı_tahmin)
Logistic_Regression_fpr, Logistic_Regression_tpr, _ = roc_curve(testy, Logistic_Regression_tahmin)

pyplot.plot(KNN_fpr, KNN_tpr, linestyle='--', label='KNN')
pyplot.plot(NBG_fpr, NBG_tpr, marker='.', label='Navy Bayes Gaussian')
pyplot.plot(Karar_Ağacı_fpr, Karar_Ağacı_tpr, marker='.', label='Karar Ağacı')
pyplot.plot(Logistic_Regression_fpr, Logistic_Regression_tpr, marker='.', label='Logistic Regresyon')

pyplot.xlabel('Gerçek Müşteri Kaybı Oluştu')
pyplot.ylabel('Gerçek Müşteri Kaybı Oluşmadı')

pyplot.legend()

pyplot.show()

Analizimiz için seçilen algoritma Logistic Regression olmuştur.

# 6.Sonuç Raporu
Müşteri kaybını çeşitli analizler kullanılarak önceden tahim etmek ve bu sayede işletmeyi müşteri kaybı oluşmadan uyarmak mümkündür.<br>
Analiz ettiğimiz veri tabanı için seçilen en uygun algoritma Logistic Regresyondur.<br>
Lojistik regresyon analizimizin sonucunda %81,3 başarılı tahmin skoru elde ettik.<br>
Müşteri kaybına sebep olan temel faktör Toplam ödemelerin yüksekliğidir.<br>
Toplam ödemeleri abonelik süresi ve aylık ödemeler takip ediyor.Çevrimiçi güvenlik ve teknik destek hizmeti almayan müşterilerinde kayıp oranının daha yüksek olduğunu söyleyebiliriz.
